In [3]:
import numpy as np
import re
import time
from stanfordcorenlp import StanfordCoreNLP
import torch
from torch.autograd import Function
from torch.autograd import Variable
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tqdm import tqdm
import time
import math
import logging
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader
from sklearn import preprocessing
from random import shuffle
BATCH_SIZE = 128
tl.set_backend('pytorch')
help(tl.set_backend)

Using numpy backend.


Help on function set_backend in module tensorly:

set_backend(backend_name)
    Sets the backend for TensorLy
    
        The backend will be set as specified and operations will used that backend
    
    Parameters
    ----------
    backend_name : {'mxnet', 'numpy', 'pytorch', 'tensorflow', 'cupy'}, default is 'numpy'



Using pytorch backend.


In [4]:
tag = [72, 71, 84, 54, 57, 65, 64]
Y = np.zeros(467)
print(Y.shape)
Y[0 : tag[0]] = 0
Y[tag[0] : tag[0]+tag[1]] = 1
Y[tag[0]+tag[1] : tag[0]+tag[1]+tag[2]] = 2
Y[tag[0]+tag[1]+tag[2] : tag[0]+tag[1]+tag[2]+tag[3]] = 3
Y[tag[0]+tag[1]+tag[2]+tag[3] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]] = 4
Y[tag[0]+tag[1]+tag[2]+tag[3]+tag[4] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5]] = 5
Y[tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5]+tag[6]] = 6
Y = Y.tolist()

(467,)


In [3]:
inputs = []
max_length = 0
for i in range(0,467):
    a = np.loadtxt('/Users/denhiroshi/Downloads/Embeding/ans_1.0_'+str(i)+'.csv',dtype=np.float64,delimiter=',')
    a = a.transpose(1,0)
    a = np.exp(a)
    a = np.sum(a,axis=0)
    a = np.log(a)
    inputs.append(a)

In [4]:
use = np.stack(inputs)
mean = use.mean()         #计算平均数
deviation = use.std()     #计算标准差
# 标准化数据的公式: (数据值 - 平均数) / 标准差
use = (use - mean) / deviation

In [5]:
b = np.fromfile('glove_WV.dat',dtype=np.float64).reshape((467,-1,200))

In [6]:
b.shape,use.shape

((467, 53, 200), (467, 467))

In [7]:
inputs = []
for i in range(467):
    inputs.append((torch.from_numpy(b[i]),torch.from_numpy(use[i]),Y[i]))
shuffle(inputs)
flag = int(len(inputs)*0.7)
train = inputs[:flag]
test = inputs[flag:]

In [8]:
t_s = []
t_e = []
t_y = []
for E,S,Y in test:
    t_s.append(S)
    t_e.append(E)
    t_y.append(Y)
t_s = torch.stack(t_s)
t_e = torch.stack(t_e)
t_y = torch.from_numpy(np.array(t_y))
t_s.shape, t_e.shape, t_y.shape

(torch.Size([141, 467]), torch.Size([141, 53, 200]), torch.Size([141]))

# glove_rnn_max: 0.6524822695035462 65轮

In [17]:
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

class RNN_max(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.GRU(
            input_size = 200,       # rnn input
            hidden_size = 64,     # rnn hidden unit
            num_layers = 1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
        )
        self.out=torch.nn.Linear(in_features=64,out_features=7)
        self.maxpool = torch.nn.MaxPool2d((53,1))

    def forward(self,sen,x):
        output,h_n=self.rnn(x.float())
        output_in_last_timestep = self.maxpool(output)
        output_in_last_timestep = output_in_last_timestep.reshape((output_in_last_timestep.shape[0],-1))
        x=self.out(output_in_last_timestep)
        return x

In [18]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
)
net = RNN_max()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
for epoch in range(1,201): 
    for step, (batch_x, sen, batch_y) in enumerate(loader):
        pred = net(sen, batch_x)
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 2: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_s, t_e)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss}")

RNN_max(
  (rnn): GRU(200, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=7, bias=True)
  (maxpool): MaxPool2d(kernel_size=(53, 1), stride=(53, 1), padding=0, dilation=1, ceil_mode=False)
)
1: accuracy:0.2978723404255319 loss: 1.8562443256378174
2: accuracy:0.4326241134751773 loss: 1.6639606952667236
3: accuracy:0.5106382978723404 loss: 1.3970987796783447
4: accuracy:0.475177304964539 loss: 1.1561895608901978
5: accuracy:0.5460992907801419 loss: 0.9396651983261108
6: accuracy:0.5815602836879432 loss: 0.5332620739936829
7: accuracy:0.6028368794326241 loss: 0.4410622715950012
8: accuracy:0.5957446808510638 loss: 0.32944566011428833
9: accuracy:0.6099290780141844 loss: 0.31363824009895325
10: accuracy:0.6028368794326241 loss: 0.16599880158901215
11: accuracy:0.5957446808510638 loss: 0.10138383507728577
12: accuracy:0.5886524822695035 loss: 0.06081625446677208
13: accuracy:0.5815602836879432 loss: 0.040260542184114456
14: accuracy:0.6028368794326241 loss: 0.022472718730

135: accuracy:0.5815602836879432 loss: 0.00027812548796646297
136: accuracy:0.5815602836879432 loss: 0.00027411323389969766
137: accuracy:0.5815602836879432 loss: 0.0002513545041438192
138: accuracy:0.5815602836879432 loss: 0.00033311161678284407
139: accuracy:0.5815602836879432 loss: 0.00028166771517135203
140: accuracy:0.5815602836879432 loss: 0.0002844742266461253
141: accuracy:0.5815602836879432 loss: 0.0002581528387963772
142: accuracy:0.5815602836879432 loss: 0.0002590111398603767
143: accuracy:0.5815602836879432 loss: 0.0002612318203318864
144: accuracy:0.5815602836879432 loss: 0.00039308410487137735
145: accuracy:0.5815602836879432 loss: 0.0002282687637489289
146: accuracy:0.5815602836879432 loss: 0.00028664726414717734
147: accuracy:0.5815602836879432 loss: 0.00037805011379532516
148: accuracy:0.5815602836879432 loss: 0.00023583004076499492
149: accuracy:0.5815602836879432 loss: 0.0002475670480635017
150: accuracy:0.5815602836879432 loss: 0.00027078218408860266
151: accuracy:0

# glove_Bi-rnn_max: 0.6808510638297872 7轮

In [20]:
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

class Bi_RNN_max(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.GRU(
            input_size = 200,       # rnn input
            hidden_size = 64,     # rnn hidden unit
            num_layers = 1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
            bidirectional=True
        )
        self.out=torch.nn.Linear(in_features=128,out_features=7)
        self.maxpool = torch.nn.MaxPool2d((53,1))

    def forward(self,sen,x):
        output,h_n=self.rnn(x.float())
        output_in_last_timestep = self.maxpool(output)
        output_in_last_timestep = output_in_last_timestep.reshape((output_in_last_timestep.shape[0],-1))
        x=self.out(output_in_last_timestep)
        return x

In [23]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
)
net = Bi_RNN_max()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
for epoch in range(1,201): 
    for step, (batch_x, sen, batch_y) in enumerate(loader):
        pred = net(sen, batch_x)
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 2: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_s, t_e)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss}")

Bi_RNN_max(
  (rnn): GRU(200, 64, batch_first=True, bidirectional=True)
  (out): Linear(in_features=128, out_features=7, bias=True)
  (maxpool): MaxPool2d(kernel_size=(53, 1), stride=(53, 1), padding=0, dilation=1, ceil_mode=False)
)
1: accuracy:0.22695035460992907 loss: 1.9261858463287354
2: accuracy:0.41134751773049644 loss: 1.567962646484375
3: accuracy:0.5815602836879432 loss: 1.2670047283172607
4: accuracy:0.6312056737588653 loss: 0.9607715010643005
5: accuracy:0.624113475177305 loss: 0.6432244181632996
6: accuracy:0.6595744680851063 loss: 0.4268020987510681
7: accuracy:0.6808510638297872 loss: 0.3015275001525879
8: accuracy:0.6312056737588653 loss: 0.15083907544612885
9: accuracy:0.624113475177305 loss: 0.11364524066448212
10: accuracy:0.6382978723404256 loss: 0.05991930142045021
11: accuracy:0.6312056737588653 loss: 0.030624818056821823
12: accuracy:0.6524822695035462 loss: 0.022307030856609344
13: accuracy:0.6524822695035462 loss: 0.010035010986030102
14: accuracy:0.66666666666

134: accuracy:0.6666666666666666 loss: 0.00013349397340789437
135: accuracy:0.6595744680851063 loss: 0.0001453535951441154
136: accuracy:0.6524822695035462 loss: 0.00015350068861152977
137: accuracy:0.6595744680851063 loss: 0.0001294067915296182
138: accuracy:0.6595744680851063 loss: 0.00013213838974479586
139: accuracy:0.6524822695035462 loss: 0.00012712478928733617
140: accuracy:0.6595744680851063 loss: 0.00013387543731369078
141: accuracy:0.6595744680851063 loss: 0.00012803077697753906
142: accuracy:0.6524822695035462 loss: 0.00011979512055404484
143: accuracy:0.6595744680851063 loss: 0.00011410032311687246
144: accuracy:0.6524822695035462 loss: 0.00011839185754070058
145: accuracy:0.6524822695035462 loss: 0.00010876655869651586
146: accuracy:0.6524822695035462 loss: 0.00013625281280837953
147: accuracy:0.6595744680851063 loss: 0.00012610299745574594
148: accuracy:0.6524822695035462 loss: 0.00011950901534873992
149: accuracy:0.6595744680851063 loss: 0.00011908667511306703
150: accur

# glove:0.44680851063829785

In [20]:
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

class RNN_old(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.GRU(
            input_size = 200,       # rnn input
            hidden_size = 64,     # rnn hidden unit
            num_layers = 1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
        )
        self.out=torch.nn.Linear(in_features=64,out_features=7)

    def forward(self,sen,x):
        output,h_n=self.rnn(x.float())
        output_in_last_timestep=h_n[-1,:,:]
        x=self.out(output_in_last_timestep)
        return x

In [ ]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
)
net = RNN_old()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
for epoch in range(1,501): 
    for step, (batch_x, sen, batch_y) in enumerate(loader):
        pred = net(sen, batch_x)
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 2: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_s, t_e)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss}")

RNN_old(
  (rnn): GRU(200, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=7, bias=True)
)
1: accuracy:0.2127659574468085 loss: 1.9432930946350098
2: accuracy:0.12056737588652482 loss: 1.965213656425476
3: accuracy:0.2127659574468085 loss: 1.9340920448303223
4: accuracy:0.12056737588652482 loss: 1.9321993589401245
5: accuracy:0.12056737588652482 loss: 1.933420181274414
6: accuracy:0.22695035460992907 loss: 1.9249428510665894
7: accuracy:0.22695035460992907 loss: 1.9938651323318481
8: accuracy:0.24113475177304963 loss: 1.9096006155014038
9: accuracy:0.14184397163120568 loss: 1.9152500629425049
10: accuracy:0.18439716312056736 loss: 1.8684868812561035
11: accuracy:0.24113475177304963 loss: 1.844362497329712
12: accuracy:0.2127659574468085 loss: 1.7581762075424194
13: accuracy:0.24113475177304963 loss: 1.7458086013793945
14: accuracy:0.28368794326241137 loss: 1.542167067527771
15: accuracy:0.3333333333333333 loss: 1.5530861616134644
16: accuracy:0.2695035460992908 loss:

138: accuracy:0.5035460992907801 loss: 0.0003175258752889931
139: accuracy:0.5035460992907801 loss: 0.00028664726414717734
140: accuracy:0.5035460992907801 loss: 0.00025294849183410406
141: accuracy:0.5035460992907801 loss: 0.00029448099667206407
142: accuracy:0.5035460992907801 loss: 0.0002509593905415386
143: accuracy:0.5106382978723404 loss: 0.00027127948123961687
144: accuracy:0.5035460992907801 loss: 0.00027157238218933344
145: accuracy:0.5035460992907801 loss: 0.00025053706485778093
146: accuracy:0.5035460992907801 loss: 0.00021967205975670367
147: accuracy:0.5035460992907801 loss: 0.00027350016171112657
148: accuracy:0.5035460992907801 loss: 0.00022682461712975055
149: accuracy:0.5035460992907801 loss: 0.00021014895173721015
150: accuracy:0.5035460992907801 loss: 0.0002713067224249244
151: accuracy:0.5035460992907801 loss: 0.00024636133457534015
152: accuracy:0.5035460992907801 loss: 0.0002392223832430318
153: accuracy:0.5035460992907801 loss: 0.0002703802892938256
154: accuracy

271: accuracy:0.46808510638297873 loss: 0.00010514940368011594
272: accuracy:0.46808510638297873 loss: 8.888244337867945e-05
273: accuracy:0.46808510638297873 loss: 0.00010717936675064266
274: accuracy:0.46808510638297873 loss: 8.746555977268144e-05
275: accuracy:0.46808510638297873 loss: 9.362356649944559e-05
276: accuracy:0.46808510638297873 loss: 9.712491737445816e-05
277: accuracy:0.46808510638297873 loss: 8.167539635905996e-05
278: accuracy:0.46808510638297873 loss: 0.00010925020615104586
279: accuracy:0.46808510638297873 loss: 0.00010566711716819555
280: accuracy:0.475177304964539 loss: 0.00011508123861858621
281: accuracy:0.475177304964539 loss: 7.611683395225555e-05
282: accuracy:0.475177304964539 loss: 7.167543662944809e-05
283: accuracy:0.475177304964539 loss: 8.178438292816281e-05
284: accuracy:0.475177304964539 loss: 7.700239075347781e-05
285: accuracy:0.475177304964539 loss: 9.832382056629285e-05
286: accuracy:0.475177304964539 loss: 8.119855920085683e-05
287: accuracy:0.4

# N-new 0.6737588652482269 17轮

In [5]:
inputs = []
MAX_LENGTH = 53
for i in range(0,467):
    a = np.loadtxt('/Users/denhiroshi/Downloads/Embeding/ans_1.0_'+str(i)+'.csv',dtype=np.float64,delimiter=',')
    a = a.transpose(1,0)
    a = np.exp(a)
    a = np.sum(a,axis=1)
    a = np.log(a)
    if a.shape[0] < MAX_LENGTH:
        add = MAX_LENGTH - a.shape[0]
        adds = np.zeros(add)
        a = np.concatenate((a, adds))
    inputs.append(a)

In [6]:
use = np.stack(inputs)
mean = use.mean()         #计算平均数
deviation = use.std()     #计算标准差
# 标准化数据的公式: (数据值 - 平均数) / 标准差
use = (use - mean) / deviation

In [7]:
b = np.fromfile('glove_WV.dat',dtype=np.float64).reshape((467,-1,200))

In [8]:
b.shape,use.shape

((467, 53, 200), (467, 53))

In [9]:
inputs = []
for i in range(467):
    inputs.append((torch.from_numpy(b[i]),torch.from_numpy(use[i]),Y[i]))
shuffle(inputs)
flag = int(len(inputs)*0.7)
train = inputs[:flag]
test = inputs[flag:]

In [10]:
t_s = []
t_e = []
t_y = []
for E,S,Y in test:
    t_s.append(S)
    t_e.append(E)
    t_y.append(Y)
t_s = torch.stack(t_s)
t_e = torch.stack(t_e)
t_y = torch.from_numpy(np.array(t_y))
t_s.shape, t_e.shape, t_y.shape

(torch.Size([141, 53]), torch.Size([141, 53, 200]), torch.Size([141]))

In [11]:
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

class RNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.GRU(
            input_size = 200,       # rnn input
            hidden_size = 64,     # rnn hidden unit
            num_layers = 1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
        )
        self.out=torch.nn.Linear(in_features=65 ,out_features=7)
        self.maxpool = torch.nn.MaxPool2d((53,1))

    def forward(self,sen,x):
        output,h_n=self.rnn(x.float())
        sen = sen.reshape((sen.shape[0],sen.shape[1],1))  # (BATCH_SIZE,53,1)
        output1 = torch.bmm(torch.transpose(output,1,2).float(),sen.float())  # (BATCH_SIZE,64,1)
        output1 = torch.sigmoid(output1)
        output1 = torch.softmax(output1,1)
        output1 = torch.bmm(output,output1) # (BATCH_SIZE,53,1)
        output = torch.cat((output1,output),2) # (BATCH_SIZE,53,65)
        output = self.maxpool(output)   # (BATCH_SIZE,1,65)
        output = output.reshape((output.shape[0],-1)) # (BATCH_SIZE,7)
        x1 = self.out(output.float())
        return x1

In [15]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
    drop_last=True,
)
net = RNN()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
for epoch in range(1,201): 
    for step, (batch_x, sen, batch_y) in enumerate(loader):
        pred = net(sen, batch_x)
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 1: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_s, t_e)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss}")

RNN(
  (rnn): GRU(200, 64, batch_first=True)
  (out): Linear(in_features=65, out_features=7, bias=True)
  (maxpool): MaxPool2d(kernel_size=(53, 1), stride=(53, 1), padding=0, dilation=1, ceil_mode=False)
)
1: accuracy:0.2695035460992908 loss: 1.9541912078857422
2: accuracy:0.3900709219858156 loss: 1.772621750831604
3: accuracy:0.3333333333333333 loss: 1.606994390487671
4: accuracy:0.4326241134751773 loss: 1.4174365997314453
5: accuracy:0.5177304964539007 loss: 1.2522685527801514
6: accuracy:0.5673758865248227 loss: 1.1295371055603027
7: accuracy:0.5531914893617021 loss: 0.9108871817588806
8: accuracy:0.6028368794326241 loss: 0.7546970844268799
9: accuracy:0.574468085106383 loss: 0.5393237471580505
10: accuracy:0.6170212765957447 loss: 0.4772818088531494
11: accuracy:0.624113475177305 loss: 0.37971144914627075
12: accuracy:0.6312056737588653 loss: 0.25231868028640747
13: accuracy:0.6312056737588653 loss: 0.18982568383216858
14: accuracy:0.6382978723404256 loss: 0.11684030294418335
15: a

135: accuracy:0.6312056737588653 loss: 0.0005545839667320251
136: accuracy:0.6312056737588653 loss: 0.00046394020318984985
137: accuracy:0.6382978723404256 loss: 0.0004654601216316223
138: accuracy:0.6382978723404256 loss: 0.00037739425897598267
139: accuracy:0.6382978723404256 loss: 0.0004795156419277191
140: accuracy:0.6382978723404256 loss: 0.0004212707281112671
141: accuracy:0.6382978723404256 loss: 0.0005138963460922241
142: accuracy:0.6382978723404256 loss: 0.00042288750410079956
143: accuracy:0.6382978723404256 loss: 0.00043049082159996033
144: accuracy:0.6382978723404256 loss: 0.000390518456697464
145: accuracy:0.6382978723404256 loss: 0.0003867931663990021
146: accuracy:0.6382978723404256 loss: 0.00042531639337539673
147: accuracy:0.6382978723404256 loss: 0.00036405399441719055
148: accuracy:0.6382978723404256 loss: 0.00033909454941749573
149: accuracy:0.6382978723404256 loss: 0.00040668994188308716
150: accuracy:0.6382978723404256 loss: 0.00033704936504364014
151: accuracy:0.

# N-new-bi 0.7163120567375887 13轮

In [16]:
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

class Bi_RNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.GRU(
            input_size = 200,       # rnn input
            hidden_size = 64,     # rnn hidden unit
            num_layers = 1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
            bidirectional=True
        )
        self.out=torch.nn.Linear(in_features=129 ,out_features=7)
        self.maxpool = torch.nn.MaxPool2d((53,1))

    def forward(self,sen,x):
        output,h_n=self.rnn(x.float())
        sen = sen.reshape((sen.shape[0],sen.shape[1],1))  # (BATCH_SIZE,53,1)
        output1 = torch.bmm(torch.transpose(output,1,2).float(),sen.float())  # (BATCH_SIZE,64,1)
        output1 = torch.sigmoid(output1)
        output1 = torch.softmax(output1,1)
        output1 = torch.bmm(output,output1) # (BATCH_SIZE,53,1)
        output = torch.cat((output1,output),2) # (BATCH_SIZE,53,65)
        output = self.maxpool(output)   # (BATCH_SIZE,1,65)
        output = output.reshape((output.shape[0],-1)) # (BATCH_SIZE,7)
        x1 = self.out(output.float())
        return x1

In [19]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
    drop_last=True,
)
net = Bi_RNN()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
for epoch in range(1,201): 
    for step, (batch_x, sen, batch_y) in enumerate(loader):
        pred = net(sen, batch_x)
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 1: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_s, t_e)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss}")

Bi_RNN(
  (rnn): GRU(200, 64, batch_first=True, bidirectional=True)
  (out): Linear(in_features=129, out_features=7, bias=True)
  (maxpool): MaxPool2d(kernel_size=(53, 1), stride=(53, 1), padding=0, dilation=1, ceil_mode=False)
)
1: accuracy:0.19148936170212766 loss: 1.9885358810424805
2: accuracy:0.475177304964539 loss: 1.713677167892456
3: accuracy:0.5106382978723404 loss: 1.5415892601013184
4: accuracy:0.574468085106383 loss: 1.284982442855835
5: accuracy:0.6524822695035462 loss: 1.1376138925552368
6: accuracy:0.6595744680851063 loss: 0.8694663643836975
7: accuracy:0.6382978723404256 loss: 0.6222941875457764
8: accuracy:0.6808510638297872 loss: 0.5021540522575378
9: accuracy:0.6737588652482269 loss: 0.351982980966568
10: accuracy:0.6666666666666666 loss: 0.27762582898139954
11: accuracy:0.6666666666666666 loss: 0.15613821148872375
12: accuracy:0.6879432624113475 loss: 0.11653878539800644
13: accuracy:0.6808510638297872 loss: 0.07489372789859772
14: accuracy:0.7092198581560284 loss: 

KeyboardInterrupt: 